In [2]:
#pip install biopandas

  Using cached biopandas-0.2.5-py2.py3-none-any.whl (263 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install biopython --upgrade

  Using cached biopython-1.76-cp36-cp36m-manylinux1_x86_64.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xmltodict

  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Preamble 

import sherlockml.datasets as sfs
import pandas as pd
import random

#sfs.get('/input/spotfunc.py', 'spotfunc.py')
#sfs.get('/input/playlists_ids_and_titles.csv', 'playlists_ids_and_titles.csv')
#sfs.get('/input/new artists2015onwards.csv', 'newartists2015onwards.csv')
#sfs.get('/input/cleaned_data.csv', 'cleaned_data.csv')

# Add more stuff here as necessary 
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

#Import all required libraries
import pandas as pd

In [8]:
from Bio import Entrez
import xmltodict

In [ ]:
##########################

In [5]:
#import xml.etree.ElementTree as ET

In [ ]:
########################

### Step 1: Find Referenc List for PMID

<div class="alert alert-warning">
<b>ACTION: Make it work for a list of PMIDs </b> 

Should work for a lists of 5000 PMIDs. Should give out a dataframe with a row with the original PMIDs and the references in the columns.

</div>

In [13]:
Entrez.email = "anina.carstens@gmx.de"
def get_links_id(pmid):
    link_list = []
    links = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pubmed_refs")
    record = Entrez.read(links)

    records = record[0][u'LinkSetDb'][0][u'Link']

    for link in records:
        link_list.append(link[u'Id'])

    return link_list

In [14]:
x = get_links_id(26816981)
print(x)

['24803944', '24731885', '24701238', '24175774', '24125973', '23886196', '23874588', '23150723', '23142798', '23036669', '23033982', '22211126', '21875474', '21875285', '21430075', '21133622', '20354574', '18383896', '17089041', '15154663', '15015612', '14626183', '11489752']


In [ ]:
##################

### Step 2: Find Author Names of References

<div class="alert alert-warning">
<b>ACTION: Make it work for approximately 500,000 PMIDs.  </b> 

Should work for 500,000 PMIDs. Give out DataFrame with multiindex. Right now only working for a single PDMI at a time

</div>

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import os
import time

In [22]:

class PubMedScraper(object):
    
    def __init__(self,
                 proxy = None):
        
        
        self.proxies = {'https' : proxy}
        self.base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
        self.db = 'pubmed'
        self.retmode = 'xml'
    
    def __query_builder(self,
                      input_keywords):
        return input_keywords.replace(' ','+')
    
    def search(self,
               query,
               max_results=15000):
        
        if ' ' in query:
            query = self.__query_builder(query)
        
        search_url = self.base + 'esearch.fcgi?db=' + self.db + '&retmode=' + \
        self.retmode + '&retmax=' + str(max_results) + '&term=' + query + \
        '&sort=relevance'+'&usehistory=y'

        search_response = requests.get(search_url,proxies = self.proxies)
        
        time.sleep(5)
        
        soup = bs(search_response.text, 'xml')
        
        webenv = soup.find('WebEnv').text
        qkey = soup.find('QueryKey').text
        article_count = int(soup.find('RetMax').text)
        
        if article_count > 10000:
            
            self.results = list()
            
            for retmin in range(0,article_count,10000):

                if (retmin + 10000) > article_count:
                    retmax = article_count
                    print('Making an API request for articles numbered ' + str(retmin)+ ' to ' + str(retmax))
                else:
                    retmax = retmin + 9999
                    print('Making an API request for articles numbered ' + str(retmin)+ ' to ' + str(retmax))
                    
                fetch_url = self.base + 'efetch.fcgi?db=' + self.db + \
                '&query_key=' + qkey + '&WebEnv=' + webenv +'&retmin=' + \
                str(retmin) +'&retmax='+\
                str(retmax)+'&rettype=xml&retmode=xml'
                
                fetch_response = requests.get(fetch_url, proxies = self.proxies)
                
                time.sleep(5)
                
                self.results.append(bs(fetch_response.text, 'xml'))
                
            
        else:
            
            fetch_url = self.base + 'efetch.fcgi?db=' + self.db + \
            '&query_key=' + qkey + '&WebEnv=' + webenv + '&retmax='+\
            str(article_count)+'&rettype=xml&retmode=xml'
            
            fetch_response = requests.get(fetch_url, proxies = self.proxies)
            
            fetch_soup = bs(fetch_response.text, 'xml')
            
            self.results = [fetch_soup]

    
    def __content_extract(self,
                       article):
        
        contents = list()
        
        contents.append(article.find('ArticleTitle').text) #Title
        contents.append(article.find('PMID').text)
        try:
            contents.append(article.find('ArticleDate').find('Year').text + '-' +\
            article.find('ArticleDate').find('Month').text + '-' +\
            article.find('ArticleDate').find('Day').text)
        except:
            try:
                contents.append(article.find('History').find('Year').text + '-' +\
                article.find('History').find('Month').text + '-' +\
                article.find('History').find('Day').text)
            except:
                contents.append('')
        
        author_list = article.find_all('Author')
        if len(author_list) == 0:
            contents.append('')
            contents.append('')
            contents.append('')
            contents.append('')
            
        elif len(author_list) == 1:
            try:
                fname = author_list[0].find('ForeName').text
            except:
                fname = ''
            try:
                lname = author_list[0].find('LastName').text
            except:
                lname = ''
            contents.append(fname + ' ' + lname)
            
            try:
                contents.append(author_list[0].find('Affiliation').text)
            except:
                contents.append('')
            contents.append('')
            contents.append('')
            
        else:
            try:
                fname = author_list[0].find('ForeName').text
            except:
                fname = ''
            try:
                lname = author_list[0].find('LastName').text
            except:
                lname = ''
            contents.append(fname + ' ' + lname)
            
            try:
                contents.append(author_list[0].find('Affiliation').text)
            except:
                contents.append('')
            authors = ''
            insts = ''
            for i,author in enumerate(author_list):
                if i == 0:
                    continue
                try:
                    authors += author.find('ForeName').text + ' ' + \
                    author.find('LastName').text + ', '
                except:
                    pass
                try:
                    insts += author.find('Affiliation').text + '_ '
                except:
                    pass
            contents.append(authors.strip(', '))
            contents.append(insts.strip(', '))
        
        keywords = article.find_all('Keyword')
        if len(keywords) >= 1:
            contents.append(', '.join([kw.text for kw in keywords]))
        else:
            contents.append('')
        
        contents.append(article.find('Journal').find('Title').text)
        
        contents.append(' '.join([abst.text for abst in article.find_all('AbstractText')]).replace('\n',''))
        
        contents = ['"' + content.replace('"','').replace('\n','').strip() + '"' for content in contents]
        return contents
            
       
    
    def saveas(self,
               file = '',
               filename=''):
        
        ### Still broken, needs fixing
        if file == 'xml':
            for result in self.results:
                with open(filename,'w') as f:
                    f.write("\n".join(str(result,encoding='utf-8') ))
        ###
        if file == 'csv':
            article_line = list()
            
            for result in self.results:
                for article in result.find_all('PubmedArticle'):
                    article_line.append(self.__content_extract(article))
                    
            with open(filename, 'w',encoding='utf-8') as f:
                f.write('"Article Title","PMID", "Published Date","Main Author","Main Author Affiliation","Secondary Author(s)","Secondary Author(s) Affiliation","Keywords","Journal Name","Abstract"\n')
                for row in article_line:
                    f.write(','.join(row) + '\n')

In [23]:
scraper = PubMedScraper()
scraper.search(query = '26816981', max_results = 1)

scraper.saveas(file = 'csv', filename = 'Test2')


In [24]:
%%time
# Read in sampled data
data = pd.read_csv('Test2')
data

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 9.54 ms


,Article Title,PMID,"""Published Date""",Main Author,Main Author Affiliation,Secondary Author(s),Secondary Author(s) Affiliation,Keywords,Journal Name,Abstract
0,Phytomedicine in the Treatment of Cancer: A He...,26816981,2015-12-01,Tanushree Chaudhary,"HTA Fellow, Centre for Health Technology Asses...","Akriti Chahar, Jitendar Kumar Sharma, Kirandee...","Research Associate, Health Technology Assessme...","Health economics, Herbal medicine",Journal of clinical and diagnostic research : ...,Cancer is reported to cause about 0.4 million ...


In [ ]:
######### Schleife einfügen

In [ ]:
%%time

from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

########################################################################################
#insert timeframe of interest
start_date = date(2019,1,1)
end_date = date(2020,1,1)
########################################################################################

appended_data = []
appended_colnames = []

for single_date in daterange(start_date, end_date):
    #full date
    run_time = str(single_date.strftime('%Y-%m-%d'))
    
    #date split into year, month, day
    year_run = int(single_date.strftime('%Y'))
    month_run = int(single_date.strftime('%m'))
    day_run = int(single_date.strftime('%d'))
    
    #apply function to calculate 52 week high
    df_final = final_func(year_run,month_run,day_run, data)
    
    #append dataframe w/ 52 week highs for each day
    appended_data.append(df_final)
    
    #append list of dates (aka column names)
    appended_colnames.append(run_time)

#merge dataframes    
appended_data = pd.concat(appended_data,axis=1, sort=False)
#rename columns with dates
appended_data.columns = appended_colnames

#showcase final resulrs
appended_data